In [1]:
#copy files from s3 ( to be done only once)
#!aws s3 cp s3://ncw210data/NewClusterFiles/ /home/ubuntu/data/models_data/ --recursive --only-show-errors

In [8]:
!head -1 /home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_7_5.csv

appl_doc_number,appl_country,appl_date,pub_doc_number,pub_date,number_of_days,invention_title,abstract,claim_text,claim_text_stemmed,top_cluster,pr_0,pr_1,pr_2,pr_3,pr_4,pr_5,pr_6,pr_7,pr_8,pr_9,count,patent_number,patent_date,patent_doc_kind,pat_number_of_days,length_of_grant,references_count,number_of_claims


In [25]:
%%writefile patent_search_script.py
# same cleaning and stemming function used while creating the csv files from xml to be used for query
import nltk
import re
sb_stemmer = nltk.stem.SnowballStemmer('english')

def tokenize_and_stem(text):
    text_stem = ''
    for word in text.lower().split():
        if text_stem:
            text_stem =  text_stem + ' ' + sb_stemmer.stem(word)
        else:
            text_stem = sb_stemmer.stem(word)
    return text_stem


def clean_stem(string):
    stopwords = {'back', 'thru', 'eg', 'hereafter', 'too', 'part', 'which', 'will', 'be', 'thereupon', 'about', 'nevertheless', 'therein', 'through', 'we', 'among', 'in', 'then', 'former', 'via', 'below', 'whereafter', 'due', 'you', 'bill', 'forty', 'few', 'not', 'with', 'rather', 'next', 'nine', 'me', 'its', 'sometime', 'yours', 'who', 'whoever', 'down', 'some', 'such', 'thereafter', 'hasnt', 'fifteen', 'both', 'as', 'ever', 'could', 'find', 'hence', 'something', 'a', 'there', 'mostly', 'whereas', 'many', 'serious', 'can', 'indeed', 'afterwards', 'whenever', 'by', 'becomes', 'may', 'after', 'couldnt', 'seemed', 'anyhow', 'etc', 'might', 'already', 'no', 'please', 'them', 'myself', 'therefore', 'from', 'along', 'ltd', 'against', 'everywhere', 'amoungst', 'because', 'where', 'sixty', 'ie', 'although', 'sincere', 'move', 'seeming', 'or', 'wherever', 'inc', 'whatever', 'into', 'anywhere', 'around', 'nor', 'see', 'several', 'sometimes', 'for', 'interest', 'beyond', 'whether', 'detail', 'describe', 'moreover', 'nobody', 'whereupon', 're', 'without', 'an', 'ours', 'perhaps', 'only', 'five', 'towards', 'keep', 'eleven', 'one', 'other', 'any', 'otherwise', 'except', 'that', 'cannot', 'behind', 'ourselves', 'under', 'within', 'fifty', 'across', 'if', 'thus', 'per', 'wherein', 'here', 'empty', 'co', 'still', 'whole', 'how', 'off', 'to', 'yourself', 'call', 'cry', 'four', 'so', 'she', 'take', 'their', 'been', 'now', 'even', 'mill', 'what', 'another', 'namely', 'always', 'themselves', 'almost', 'six', 'formerly', 'ten', 'found', 'onto', 'yet', 'between', 'give', 'hers', 'herein', 'eight', 'above', 'anyway', 'third', 'himself', 'front', 'over', 'two', 'much', 'latter', 'itself', 'besides', 'those', 'on', 'twenty', 'up', 'us', 'amongst', 'beforehand', 'but', 'most', 'same', 'mine', 'should', 'this', 'full', 'herself', 'her', 'thick', 'con', 'everything', 'is', 'am', 'three', 'throughout', 'again', 'enough', 'your', 'once', 'hereupon', 'become', 'yourselves', 'everyone', 'before', 'i', 'whereby', 'others', 'must', 'seems', 'elsewhere', 'were', 'either', 'would', 'became', 'hundred', 'toward', 'very', 'latterly', 'top', 'often', 'beside', 'cant', 'else', 'the', 'however', 'and', 'somehow', 'him', 'noone', 'somewhere', 'our', 'nothing', 'de', 'fill', 'well', 'it', 'all', 'last', 'do', 'these', 'has', 'upon', 'every', 'side', 'system', 'put', 'thence', 'twelve', 'becoming', 'show', 'un', 'least', 'of', 'have', 'own', 'since', 'though', 'whither', 'out', 'hereby', 'meanwhile', 'none', 'while', 'whom', 'further', 'why', 'made', 'whose', 'my', 'someone', 'they', 'during', 'anyone', 'first', 'go', 'less', 'his', 'anything', 'thereby', 'amount', 'together', 'never', 'was', 'thin', 'also', 'each', 'fire', 'are', 'when', 'alone', 'had', 'until', 'done', 'more', 'at', 'than', 'nowhere', 'seem', 'whence', 'name', 'neither', 'he', 'get', 'being', 'bottom'}
    #strip and change to lower case and replace commas and semi colons with spaces
    stem = string.strip().lower().replace(';', ' ').replace(',', ' ').replace(':', ' ').replace('(',' ').replace(')',' ').replace('#', ' ').replace('.', ' ').strip()
         
    # remove words that only have numbers( second one removes special characters also)
    stem = re.sub(r'\b\d+\b', ' ',stem).strip()
    
    #remove special characters at the end of words
    stem = re.sub(r'([^\w\s]|_)+(?=\s|$)', ' ',stem).strip()
    
    #remove any words that have a number in it ( even if in the middle )
    stem = re.sub(r'\w*\d\w*', ' ',stem).strip()
    
    # remove any words with only one alphabet
    stem = re.sub(r'\b[a-zA-Z]\b', ' ',stem).strip()
    
    # remove stop words and change to lower case
    stem = ' '.join([item for item in (stem.strip().split()) if item not in stopwords])
    
    # get words greater than length 4 and less than 25
    stem = ' '.join([item for item in (re.findall('\w{4,25}', stem))])
    #re.findall('\w{4,25}', stem).join(' ').strip() 
    
     #apply stemming for all words
    stem = tokenize_and_stem(stem)
    return stem

#query for first level cluster and then second level cluster
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction import text
import pandas as pd
from scipy import spatial
import nltk
from string import printable
sb_stemmer = nltk.stem.SnowballStemmer('english')

my_words = ['','()','(),']
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_words)

def tokenize(doc):
    return doc.lower().split(" ")

def query_first_level_cluster(query_string):
    #find first level cluster
    nmf_filename = '/home/ubuntu/data/models_data/nmfmodels/nmf_model_X.sav'
    tfidf_filename = '/home/ubuntu/data/models_data/tfidfmodels/tfidf_model_X.sav'
    # load the model from disk
    loaded_nmf_model = pickle.load(open(nmf_filename, 'rb'))
    loaded_tfidf_model = pickle.load(open(tfidf_filename, 'rb'))

    query_string_pr_stem = clean_stem(query_string)

    query_tfidf = loaded_tfidf_model.transform([query_string_pr_stem])
    nmf_query = loaded_nmf_model.transform(query_tfidf)
    # print(nmf_query)
    # nmf_query.argmax(axis=1)
    return(nmf_query.argmax(axis=1)[0])

def query_second_level_cluster(query_string, first_level_cluster):
    #find second level cluster
    nmf_filename = '/home/ubuntu/data/models_data/nmfmodels/nmf_model_' + str(first_level_cluster) + '.sav'
    tfidf_filename = '/home/ubuntu/data/models_data/tfidfmodels/tfidf_model_' + str(first_level_cluster) + '.sav' 
    # load the model from disk
    loaded_nmf_model = pickle.load(open(nmf_filename, 'rb'))
    loaded_tfidf_model = pickle.load(open(tfidf_filename, 'rb'))
    
    query_string_pr_stem = clean_stem(query_string)
    
    query_tfidf = loaded_tfidf_model.transform([query_string_pr_stem])
    nmf_query = loaded_nmf_model.transform(query_tfidf)
    # print(nmf_query)
    # nmf_query.argmax(axis=1)
    return(nmf_query.argmax(axis=1)[0])

def query_similarities(query_string, first_level_cluster,second_level_cluster,cosine_sim_threshold,top):
    #find top similarities for a given cluster combo
    
    filename = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_' + str(first_level_cluster) + '_' + str(second_level_cluster) + '.csv'
    tfidf_model_filename = '/home/ubuntu/data/models_data/tfidfmodels/tfidf_model_' + str(first_level_cluster) + '_' + str(second_level_cluster) + '.sav'
    tfidf_filename = '/home/ubuntu/data/models_data/tfidfmodels/tfidf_' + str(first_level_cluster) + '_' + str(second_level_cluster) + '.sav'

    df_data = pd.read_csv(filename,usecols = ['appl_doc_number', 'appl_doc_number', 'invention_title','abstract','claim_text','patent_number'])
    loaded_tfidf_vectorizer = pickle.load(open(tfidf_model_filename, 'rb'))
    loaded_tfidf = pickle.load(open(tfidf_filename, 'rb'))
    
    
    query_tfidf = loaded_tfidf_vectorizer.transform([query_string])

    df_result = pd.DataFrame(columns=['cosine_similarity', 'appl_doc_number','invention_title','abstract','claim_text','patent_number'])

    # get the result and add to a new dataframe
    loc_index = 0
    patent_query = query_tfidf.toarray()[0]
    for index_corpus, patent_corpus in enumerate(loaded_tfidf.toarray()):
        #cosine_sim = cosine_similarity(patent_query, patent_corpus)
        cosine_sim = 1 - spatial.distance.cosine(patent_query, patent_corpus)
        if cosine_sim > cosine_sim_threshold:
            #res_row = df_data.iloc[index_corpus]
            df_result.loc[loc_index] = [cosine_sim,df_data.iloc[index_corpus]['appl_doc_number'],df_data.iloc[index_corpus]['invention_title'],df_data.iloc[index_corpus]['abstract'],df_data.iloc[index_corpus]['claim_text'],df_data.iloc[index_corpus]['patent_number']]
            #df_result.loc[loc_index] = [cosine_sim,res_row['appl_doc_number'],res_row['invention_title'],res_row['abstract'],res_row['claim_text'],res_row['patent_number']]
            
            loc_index += 1

    # sort the results and pick top results        
    df_result = df_result.sort_values('cosine_similarity',ascending=[False])
       
    df_result = df_result[:top]
    
    #remove any non-printable characters
    df_result['abstract'] = df_result['abstract'].apply(lambda x: re.sub(r'[^\x00-\x7f]',r'', x))
    df_result['abstract'] = df_result['abstract'].apply(lambda x: x.replace(" &#8216", '').replace(" &#8217", ''))
    df_result['claim_text'] = df_result['claim_text'].apply(lambda x: re.sub(r'[^\x00-\x7f]',r'', x))
    df_result['claim_text'] = df_result['claim_text'].apply(lambda x: x.replace("&#8216;", ' ').replace("&#8217;", ' '))
 
    return(df_result)

def query_wrapper(query_string,cosine_sim_threshold,top):
    first_cluster = query_first_level_cluster(query_string)
    second_cluster = query_second_level_cluster(query_string,first_cluster)
    df_result = query_similarities(query_string,first_cluster,second_cluster,cosine_sim_threshold,top)
    return df_result,first_cluster,second_cluster
    #return(first_cluster,second_cluster)

Overwriting patent_search_script.py


In [26]:
#https://stackoverflow.com/questions/42309133/return-pandas-dataframe-as-jsonp-response-in-python-flask

In [27]:
# from patent_search_script import *
# print(query_wrapper('A method for controlling subscriber access in a network capable of establishing connections with a plurality of services, comprising:   receiving a communication from a subscriber using a first communication network coupled to a second communication network, said communication optionally including a domain identifier associated with a service on said second communication network; and    authorizing said subscriber to access a service on said second communication network using one of a plurality of virtual circuits, said authorizing based upon a domain configuration override attribute associated with the virtual circuit used to receive said communication from',0.3,5))

In [28]:
#https://stackoverflow.com/questions/7023052/configure-flask-dev-server-to-be-visible-across-the-network    

In [29]:
%%writefile patent_api.py
from flask import request, url_for
from flask_api import FlaskAPI, status, exceptions
from flask_jsonpify import jsonpify
from patent_search_script import *
from flask import json
from flask_cors import CORS, cross_origin

app = FlaskAPI(__name__)
CORS(app)

@app.route('/patentsearch',methods=['GET'])
def patentsearch():
    if request.method == 'GET':
        query_string = str(request.args.get('query_string'))
        cosine_sim_threshold = float(request.args.get('cosine_sim_threshold'))
        top = int(request.args.get('top'))
        
        results, first_cluster, second_cluster = query_wrapper(query_string,cosine_sim_threshold,top)
    
        f_topics_filename = '/home/ubuntu/data/models_data/topics/topics_X.csv'
        s_topics_filename = '/home/ubuntu/data/models_data/topics/topics_' + str(second_cluster) + '.csv'
        df_f_topics = pd.read_csv(f_topics_filename,header=None,names = ['i','topics'])
        df_s_topics = pd.read_csv(s_topics_filename,header=None,names = ['i','topics'])
        
        charts_filename = '/home/ubuntu/data/models_data/chartdatafiles/chart_cluster_data_' + str(first_cluster) + '_' + str(second_cluster) + '.csv'
        chartfile = open(charts_filename, 'r') 
        i = 1
        chart_text = ''
        for line in chartfile:
            if i == 1:
                chart_text = "\"charts_data" + str(i) + "\":" + line 
            else:
                chart_text += ", " + "\"charts_data" + str(i) + "\":" + line 
            i += 1    
        
        json_str = '{'
        json_str += '\"searchresult\":'
        json_str += results.to_json(orient='records')
        json_str +=  ',' + "\"firstcluster\":" + "\"" + str(first_cluster) + "\""
        json_str +=  ',' + "\"secondcluster\":" + "\"" + str(second_cluster) + "\""
        json_str +=  ',' + "\"firstcluster_topics\":" + "\"" + str(df_f_topics.iloc[first_cluster]['topics']).rsplit(' ', 1900)[0] + "\""
        json_str +=  ',' + "\"secondcluster_topics\":" + "\"" + str(df_s_topics.iloc[second_cluster]['topics']).rsplit(' ', 1900)[0] + "\""
        json_str +=  ',' + "\"charts_data\": {" + str(chart_text) + "}"
        json_str += '}'
        return json_str
        
if __name__ == "__main__":
    app.run(host= '0.0.0.0',port=8080,debug=False) 
    #app.run(debug=True) 

Overwriting patent_api.py


In [14]:
#run below code in terminal to start flask
#python ./patent_api.py

#python ./patent_api.py &
26744

#!chmod +x patent_api.py
#ps -ef | grep patent_api.py
#kill -9 xxxx

26744

In [ ]:
#some sample api urls for testing

http://ec2-54-89-199-144.compute-1.amazonaws.com:8080/patentsearch?query_string='A new and distinct cultivar of ornamental  Hosta  plant named Beckoning; as herein described and illustrated, with yellowish leaf center and blue-green margin, suitable as a potted plant, for the garden, and for cut flower arrangements.'&top=5&cosine_sim_threshold=0.3
        
http://ec2-54-89-199-144.compute-1.amazonaws.com:8080/patentsearch?query_string='A polymerization process comprising:   (a) making a polymeric material in a reactor loop comprising a fluidized bed reactor comprising a feed distributor, a recycle loop associated with the fluidized bed reactor and comprising a feed line; and (b) collecting spitwads in or removing spitwads from a feed to the fluidized bed reactor and the associated feed distributor with a spitwad catcher, wherein the spitwad catcher is located outside the fluidized bed reactor and in the feed line to the fluidized bed reactor and feed distributor.'&top=5&cosine_sim_threshold=0.3        
        
http://ec2-54-89-199-144.compute-1.amazonaws.com:8080.amazonaws.com.amazonaws.com:8080/patentsearch?query_string=''&top=5&cosine_sim_threshold=0.3
        
http://ec2-52-90-56-114.compute-1.amazonaws.com.amazonaws.com:8080/patentsearch?query_string='umbrella'&top=5&cosine_sim_threshold=0.3        


http://ec2-52-90-56-114.compute-1.amazonaws.com:8080/patentsearch?query_string=%27A%20method%20for%20controlling%20subscriber%20access%20in%20a%20network%20capable%20of%20establishing%20connections%20with%20a%20plurality%20of%20services,%20comprising:%20receiving%20a%20communication%20from%20a%20subscriber%20using%20a%20first%20communication%20network%20coupled%20to%20a%20second%20communication%20network,%20said%20communication%20optionally%20including%20a%20domain%20identifier%20associated%20with%20a%20service%20on%20said%20second%20communication%20network;%20and%20authorizing%20said%20subscriber%20to%20access%20a%20service%20on%20said%20second%20communication%20network%20using%20one%20of%20a%20plurality%20of%20virtual%20circuits,%20said%20authorizing%20based%20upon%20a%20domain%20configuration%20override%20attribute%20associated%20with%20the%20virtual%20circuit%20used%20to%20receive%20said%20communication%20from%27&top=5&cosine_sim_threshold=0.3

http://ec2-54-89-199-144.compute-1.amazonaws.com:8080/patentsearch?query_string='A new and distinct cultivar of ornamental  Hosta  plant named Beckoning; as herein described and illustrated, with yellowish leaf center and blue-green margin, suitable as a potted plant, for the garden, and for cut flower arrangements.'&top=5&cosine_sim_threshold=0.3

http://ec2-52-90-56-114.compute-1.amazonaws.com:8080/patentsearch?query_string='A polymerization process comprising:   (a) making a polymeric material in a reactor loop comprising a fluidized bed reactor comprising a feed distributor, a recycle loop associated with the fluidized bed reactor and comprising a feed line; and (b) collecting spitwads in or removing spitwads from a feed to the fluidized bed reactor and the associated feed distributor with a spitwad catcher, wherein the spitwad catcher is located outside the fluidized bed reactor and in the feed line to the fluidized bed reactor and feed distributor.'&top=5&cosine_sim_threshold=0.3
        
http://ec2-52-90-56-114.compute-1.amazonaws.com:8080/patentsearch?query_string=''&top=5&cosine_sim_threshold=0.3
        
http://ec2-52-90-56-114.compute-1.amazonaws.com:8080/patentsearch?query_string=The%20polymerization%20process%20of%20claim%201%20wherein%20the%20polymeric%20material%20comprises%20a%20homopolymer%20of%20ethylene%20or%20an%20interpolymer%20of%20ethylene%20and%20at%20least%20one%20other%20olefin.%205%20.%20The%20polymerization%20process%20of%20claim%201%20wherein%20the%20spitwad%20catcher%20comprises%20a%20screen,%20filter%20or%20strainer.%206%20.%20An%20apparatus%20for%20making%20a%20polymeric%20material%20comprising:%20(a)%20a%20reactor%20loop%20comprising%20a%20fluidized%20bed%20reactor%20having%20a%20feed%20distributor,%20a%20recycle%20loop%20associated%20with%20the%20fluidized%20bed%20reactor%20and%20comprising%20a%20feed%20line,%20and%20(b)%20a%20spitwad%20catcher%20for%20collecting%20spitwads%20in%20or%20removing%20spitwads%20from%20the%20feed%20to%20the%20fluidized%20bed%20reactor%20and%20associated%20feed%20distributor,%20where%20the%20spitwad%20catcher%20is%20located%20outside%20the%20fluidized%20bed%20reactor%20and%20in%20the%20feed%20line%20to%20the%20fluidized%20bed%20reactor%20and%20feed%20distributor.%207%20.%20The%20apparatus%20of%20claim%206%20wherein%20the%20fluidized%20bed%20reactor%20is%20a%20continuous,%20gas%20phase%20reactor.%208%20.%20The%20apparatus%20of%20claim%206%20wherein%20the%20polymeric%20material%20comprises%20a%20polyolefin.&top=5&cosine_sim_threshold=0.3

In [1]:
!sed -n '1000'p /home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_7_5.csv



10990225,US,20041115,20050175028,20050811,269,Method for improving the quality of playback in the packet-oriented transmission of audio/video data,"Strong fluctuations of the transmission capacity may frequently occur in the transmission of audio/video signals in modern telecommunication networks. Occasionally, even breaks in transmission may occur. To improve playback, an exemplary method is provided, which may be used in GSM networks and future UMTS networks as well as for the fixed network. In scalable encoding methods for audio and video transmission, the bit stream is depacketized into a base bit stream and a defined number of enhancement bit streams. The exemplary method provides that the base bit stream, which may ensure a fully adequate playback, is transmitted with priority. Enhancement bit streams are transmitted only to the extent allowed by the available transmission bit rate in the network. The prioritized transmission of the base bit stream allows for playback to start qu

In [27]:
!curl -X GET 'http://ec2-54-211-237-164.compute-1.amazonaws.com:8080/patentsearch?query_string=%27A%20method%20for%20controlling%20subscriber%20access%20in%20a%20network%20capable%20of%20establishing%20connections%20with%20a%20plurality%20of%20services,%20comprising:%20receiving%20a%20communication%20from%20a%20subscriber%20using%20a%20first%20communication%20network%20coupled%20to%20a%20second%20communication%20network,%20said%20communication%20optionally%20including%20a%20domain%20identifier%20associated%20with%20a%20service%20on%20said%20second%20communication%20network;%20and%20authorizing%20said%20subscriber%20to%20access%20a%20service%20on%20said%20second%20communication%20network%20using%20one%20of%20a%20plurality%20of%20virtual%20circuits,%20said%20authorizing%20based%20upon%20a%20domain%20configuration%20override%20attribute%20associated%20with%20the%20virtual%20circuit%20used%20to%20receive%20said%20communication%20from%27&top=5&cosine_sim_threshold=0.3'

[
  [
    0.6014921979121006,
    11308622,
    "Reducing Overhead When Setting Up Multiple Virtual Circuits Using Signaling Protocols",
    "Setting up a group of virtual circuits using a single set up message request. In an embodiment, fewer than all of the virtual circuits in the group are immediately provisioned, and the remaining virtual circuits are placed in an inactive status by appropriate configuration of all the devices in the path of the group of virtual circuits. Each of the inactive virtual circuits can be activated (complete provisioning) as and when required. The bandwidth overhead on the networks is reduced as fewer signaling messages would be used in provisioning several virtual circuits. The parsing overhead is reduced on the devices processing the signaling messages as a result.",
    "1 . A method of setting up a plurality of virtual circuits between a first end system and a second end system, said plurality of virtual circuits being set up on a network connecting 